In [10]:
import pandas as pd

# Start and End Dates
start_time = '2005-01-01 00:00:00'
end_time = '2005-12-31 23:59:59'

# Generate 6-hour intervals for the entire year
timestamps = pd.date_range(start=start_time, end=end_time, freq='6H')

# Create DataFrame with All Timesteps
df = pd.DataFrame({'index': timestamps})

# Add a 'date' column to group by day
df['date'] = df['index'].dt.date  # Extract day for grouping

# Assign sequential cluster IDs per day
df_daily = df[['date']].drop_duplicates().reset_index(drop=True)
df_daily['clusters'] = range(len(df_daily))  # Sequential cluster IDs (0, 1, 2, ...)

# Map Daily Clusters back to Timesteps
df = df.merge(df_daily, on='date', how='left')

# Format 'index' column to match expected Calliope format
df['index'] = df['index'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Drop the 'date' column (not needed in the final CSV)
df = df.drop(columns=['date'])

# Save the DataFrame to CSV
df.to_csv('clusters-sequential.csv', index=False)




In [9]:
import pandas as pd

# Start and End Dates
start_time = '2005-01-01 00:00:00'
end_time = '2005-12-31 23:59:59'

# Generate 6-hour intervals for the entire year
timestamps = pd.date_range(start=start_time, end=end_time, freq='6H')

# Create DataFrame with All Timesteps
df = pd.DataFrame({'index': timestamps})
df['date'] = df['index'].dt.date  # Extract day for grouping

# Specify days to exclude (example: weekends or custom list)
# Example: Exclude weekends (Saturday and Sunday)
df['weekday'] = df['index'].dt.weekday  # 0=Monday, 6=Sunday
days_to_exclude = df[df['weekday'] >= 5]['date'].unique()  # Exclude Saturdays (5) and Sundays (6)

# Remove excluded days
df = df[~df['date'].isin(days_to_exclude)]

# Reassign Sequential Cluster IDs to Remaining Days
df_daily = df[['date']].drop_duplicates().reset_index(drop=True)
df_daily['clusters'] = range(len(df_daily))  # Sequential Cluster IDs

# Map Daily Clusters back to Timesteps
df = df.merge(df_daily, on='date', how='left')

# Format 'index' column to match Calliope expectations
df['index'] = df['index'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Drop unnecessary columns
df = df.drop(columns=['date', 'weekday'])

# Save to CSV
df.to_csv('clusters-excluded_days.csv', index=False)
